In [2]:
import numpy as np

import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim     


import time 

In [3]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')

        self.layers = layers
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)

        self.H1 = self.linears[0]
    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        #convert to float
        a = x.float()
        
        for i in range(len(self.layers)-2):
            z = self.linears[i](a)
            a = self.activation(z)
            
        a = self.linears[-1](a) 
         
        return a
    
    def forward_direct(self, x):
        
        z = x.float()
        H = self.linears[0].weight

        for i in range(len(self.layers)-2):
            L = self.linears[i](z)
            z = self.activation(L)
            G = (1-torch.square(z))*H.t() #\sigma'(L)*H
            H = torch.matmul(self.linears[i+1].weight,G.t())

        z = self.linears[-1](z)
         
        return z,H

In [4]:
layers = np.array([1,50,50,50,1])
PINN = Sequentialmodel(layers)
    

In [5]:
x = torch.tensor(5.0).view(-1,1)

start_time = time.time()
z,H = PINN.forward_direct(x)
# print("output ",z.shape,H.shape)
print("Function value",z)
print("Direct Calculation of Derivative",H)
time.time() - start_time

Function value tensor([[-0.5295]], grad_fn=<AddmmBackward0>)
Direct Calculation of Derivative tensor([[0.0003]], grad_fn=<MmBackward0>)


0.004698514938354492

In [6]:
start_time = time.time()
x.requires_grad = True
y = PINN(x)
y_x = autograd.grad(y,x,torch.ones([x.shape[0], 1]), retain_graph=True, create_graph=True,allow_unused = True)[0]
print("Autograd Calculation of Derivative",y_x)
time.time() - start_time

Autograd Calculation of Derivative tensor([[0.0003]], grad_fn=<TBackward0>)


0.30113983154296875

In [7]:
# Training data

t_train = torch.linspace(0, 5, 100).view(-1, 1)
u_train = (0.5 * (t_train **2) + t_train)

In [8]:
# Generate Collocation Points

N_f = 10000
t_f = torch.linspace(0,5,N_f).view(-1,1).requires_grad_(True)


# Generate IC Points

t_i = torch.tensor([[0.0]])

In [9]:
def compute_residuals_with_autograd():
    u_f_pred = PINN(t_f)
    u_t = autograd.grad(u_f_pred,t_f,grad_outputs=torch.ones_like(u_f_pred), create_graph=True)[0]
    
    res_pde = u_t - t_f


    u_i_pred = PINN(t_i)

    res_ic = u_i_pred
    

    return res_pde, res_ic

In [ ]:
# def compute_residuals_directly():
#     z,u_t = PINN.forward_direct(t_f)
    
    
#     res_pde = u_t - t_f


#     u_i_pred = PINN(t_i)

#     res_ic = u_i_pred
    

#     return res_pde, res_ic

In [10]:
def compute_losses_autograd():
    res_pde, res_ic = compute_residuals_with_autograd()

    loss_pde = torch.mean(res_pde**2)
    loss_ic = torch.mean(res_ic **2)

    u_pred = PINN(t_train)

    loss_data = torch.mean((u_pred - u_train)**2)

    total_loss = loss_pde + loss_ic + loss_data

    return loss_pde, loss_ic,loss_data, total_loss

In [ ]:
# def compute_losses_direct():
#     res_pde, res_ic = compute_residuals_directly()

#     loss_pde = torch.mean(res_pde**2)
#     loss_ic = torch.mean(res_ic **2)

#     u_pred = PINN(t_train)

#     loss_data = torch.mean((u_pred - u_train)**2)

#     total_loss = loss_pde + loss_ic + loss_data

#     return loss_pde, loss_ic,loss_data, total_loss

In [11]:
optimizer = torch.optim.Adam(PINN.parameters(), lr = 1e-3)

In [12]:
start_time = time.time()

num_epochs = 10000

loss_pde_list, loss_ic_list, loss_data_list, total_loss_list = [],[],[],[]

for epoch in range(num_epochs):
    optimizer.zero_grad()

    loss_pde, loss_ic, loss_data, total_loss = compute_losses_autograd()

    loss_pde.backward(retain_graph=True)
    loss_ic.backward(retain_graph=True)
    loss_data.backward()


    optimizer.step()

    loss_pde_list.append(loss_pde.item()); loss_ic_list.append(loss_ic.item()); loss_data_list.append(loss_data.item()); total_loss_list.append(total_loss.item())

    if epoch % 500 == 0:
        print(f"Epoch {epoch}: loss_pde={loss_pde.item():.3e},  loss_ic={loss_ic.item():.3e}, loss_data={loss_data.item():.3e}, total={total_loss.item():.3e}")

end_time = time.time()

print(f'Total Training Time: {(end_time - start_time): .4f}seconds')



Epoch 0: loss_pde=8.589e+00,  loss_ic=0.000e+00, loss_data=7.858e+01, total=8.716e+01
Epoch 500: loss_pde=6.703e-01,  loss_ic=2.431e-02, loss_data=1.582e-01, total=8.529e-01
Epoch 1000: loss_pde=6.447e-01,  loss_ic=2.653e-02, loss_data=1.046e-01, total=7.758e-01
Epoch 1500: loss_pde=6.438e-01,  loss_ic=2.677e-02, loss_data=1.020e-01, total=7.725e-01
Epoch 2000: loss_pde=6.437e-01,  loss_ic=2.685e-02, loss_data=1.013e-01, total=7.719e-01
Epoch 2500: loss_pde=6.437e-01,  loss_ic=2.683e-02, loss_data=1.012e-01, total=7.718e-01
Epoch 3000: loss_pde=6.437e-01,  loss_ic=2.680e-02, loss_data=1.012e-01, total=7.717e-01
Epoch 3500: loss_pde=6.470e-01,  loss_ic=2.955e-02, loss_data=9.543e-02, total=7.720e-01


KeyboardInterrupt: 

In [ ]:
# start_time = time.time()

# num_epochs = 5000

# loss_pde_list, loss_ic_list, loss_data_list, total_loss_list = [],[],[],[]

# for epoch in range(num_epochs):
#     optimizer.zero_grad()

#     loss_pde, loss_ic, loss_data, total_loss = compute_losses_direct()

#     loss_pde.backward(retain_graph=True)
#     loss_ic.backward(retain_graph=True)
#     loss_data.backward()


#     optimizer.step()

#     loss_pde_list.append(loss_pde.item()); loss_ic_list.append(loss_ic.item()); loss_data_list.append(loss_data.item()); total_loss_list.append(total_loss.item())

#     if epoch % 500 == 0:
#         print(f"Epoch {epoch}: loss_pde={loss_pde.item():.3e},  loss_ic={loss_ic.item():.3e}, loss_data={loss_data.item():.3e}, total={total_loss.item():.3e}")

# end_time = time.time()

# print(f'Total Training Time: {(end_time - start_time): .4f}seconds')

Epoch 0: loss_pde=2.837e+00,  loss_ic=3.054e-03, loss_data=5.106e-01, total=3.350e+00
Epoch 500: loss_pde=2.718e+00,  loss_ic=8.119e-03, loss_data=5.883e-01, total=3.314e+00
Epoch 1000: loss_pde=2.718e+00,  loss_ic=8.987e-03, loss_data=5.877e-01, total=3.314e+00


KeyboardInterrupt: 